1. Setup & Imports

In [1]:
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np


2. Read Raw Data from S3

In [3]:
s3_path = "s3://aws-practice-visualpath/Housing.csv" ##Provide your aws s3 path

df = wr.s3.read_csv(s3_path)

print("Raw Data Shape:", df.shape)
df.head()


Raw Data Shape: (545, 13)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


3. Basic Data Inspection

In [4]:
print("Columns:\n", df.columns)
print("\nData Types:\n", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())
print("\nDuplicate Rows:", df.duplicated().sum())


Columns:
 Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

Data Types:
 price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object

Missing Values:
 price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

Duplicate Rows: 0


4. Remove Duplicates

In [5]:
df = df.drop_duplicates().reset_index(drop=True)
print("After removing duplicates:", df.shape)


After removing duplicates: (545, 13)


5. Handle Missing Values

In [6]:
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns

df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])


6. Outlier Removal (IQR Method)

In [7]:
def remove_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
df = remove_outliers(df, numeric_cols)

print("Shape after outlier removal:", df.shape)


Shape after outlier removal: (365, 13)


7. Feature Encoding (Label Encoding + One-Hot)

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in df.select_dtypes(include=["object"]).columns:
    df[col] = label_encoder.fit_transform(df[col])


8. Feature Scaling (StandardScaler)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


9. Final Preprocessed Data

In [11]:
print(df.head())
print(df.describe())
print("Final Shape:", df.shape)


       price      area  bedrooms  bathrooms   stories  mainroad  guestroom  \
15  4.058695  0.799475  2.017626        0.0  0.725729         1          0   
20  3.777677 -0.176494  0.413196        0.0  0.725729         1          0   
22  3.693371  1.990389  0.413196        0.0 -0.899369         1          1   
27  3.496659  2.469660  0.413196        0.0 -0.899369         1          0   
40  3.075131  1.118988  0.413196        0.0  0.725729         1          0   

    basement  hotwaterheating  airconditioning   parking  prefarea  \
15         1                0                0  1.925353         0   
20         1                1                0  1.925353         0   
22         1                0                1  0.609335         0   
27         0                0                0  0.609335         0   
40         1                0                1 -0.706684         1   

    furnishingstatus  
15                 1  
20                 1  
22                 0  
27                

10. Save Cleaned Data Locally

In [12]:
df.to_csv("cleaned_data.csv", index=False)


In [14]:
wr.s3.to_csv(
    df=df,
    path="s3://aws-practice-visualpath/processed/cleaned_Housing.csv"
)

print("Uploaded cleaned file to S3.")


Uploaded cleaned file to S3.


12. Update Glue Catalog

In [16]:
wr.catalog.create_csv_table(
    database="house_price",
    table="housing_cleaned",
    path="s3://aws-practice-visualpath/processed/",
    columns_types={col: "string" for col in df.columns}  # simplify
)


13. Query Table using Athena

In [18]:
df_at = wr.athena.read_sql_query(
    sql="SELECT * FROM house_price.housing_cleaned LIMIT 20",
    database="house_price",
    ctas_approach=False
)

df_at.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,<NA>,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
1,15,4.05869499602107,0.7994747830761767,2.017626274203175,0.0,0.7257285970598193,1,0,1,0,0,1.9253528290154525,0
2,20,3.777676852951908,-0.17649406673999535,0.4131957947169902,0.0,0.7257285970598193,1,0,1,1,0,1.9253528290154525,0
3,22,3.6933714100311588,1.990389153387577,0.4131957947169902,0.0,-0.899369181632414,1,1,1,0,1,0.6093345095573248,0
4,27,3.496658709882745,2.4696595707080187,0.4131957947169902,0.0,-0.899369181632414,1,0,0,0,0,0.6093345095573248,0
